In [94]:
import random
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
from torch.utils.data import DataLoader, Dataset

class ReplayBuffer(Dataset):
    def __init__(self, capacity=10000):
        self.memory = deque(maxlen=capacity)

    def put(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def __len__(self):
        return len(self.memory)

    def __getitem__(self, idx):
        return self.memory[idx]

class Network(nn.Module):
    def __init__(self, state_size, action_size, atoms, z):
        super(Network, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.atoms = atoms
        self.z = z

        self.layers = nn.Sequential(
            nn.Linear(self.state_size, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU()
        )
        self.value_streams = nn.ModuleList([nn.Linear(64, self.atoms) for _ in range(self.action_size)])

    def forward(self, x):
        x = self.layers(x)
        return [torch.softmax(value_stream(x), dim=-1) for value_stream in self.value_streams]

class DQNAgent:
    def __init__(self, env, batch_size, target_update, v_max, v_min, atoms):
        self.env = env
        self.env.seed(1234)
        self.state_size = env.observation_space.shape[0]
        self.action_size = env.action_space.n

        self.batch_size = batch_size
        self.v_max = v_max
        self.v_min = v_min
        self.atoms = atoms
        self.delta_z = float(self.v_max - self.v_min) / (self.atoms - 1)
        self.z = torch.linspace(self.v_min, self.v_max, steps=self.atoms)

        self.dqn = Network(self.state_size, self.action_size, self.atoms, self.z)
        self.dqn_target = Network(self.state_size, self.action_size, self.atoms, self.z)
        self.optimizer = optim.Adam(self.dqn.parameters(), lr=0.001)
        self.criterion = nn.CrossEntropyLoss()
        self.memory = ReplayBuffer(capacity=10000)
        self.target_update = target_update
        self.gamma = 0.99
        self.Soft_Update = False
        self.TAU = 0.1
        self.train_start = 1000
        self._target_hard_update()

    def get_action(self, state, epsilon):
        state = torch.FloatTensor(state).unsqueeze(0)
        if random.random() <= epsilon:
            action = random.randrange(self.action_size)
        else:
            with torch.no_grad():
                z = self.dqn(state)
                z_concat = torch.cat(z, dim=1).view(-1, self.action_size, self.atoms)
                z_broadcast = self.z.view(1, 1, self.atoms)
                q = torch.sum(z_concat * z_broadcast, dim=2)
                action = q.argmax(1).item()
        return action

    def train_step(self, states, actions, rewards, next_states, dones):
        states = torch.FloatTensor(states)
        actions = torch.LongTensor(actions)
        rewards = torch.FloatTensor(rewards)
        next_states = torch.FloatTensor(next_states)
        dones = torch.FloatTensor(dones)

        with torch.no_grad():
            z = self.dqn(next_states)
            z_ = self.dqn_target(next_states)
            z_concat = np.vstack(z)
            z_broadcast = self.z.view(1, 1, self.atoms)
            q = np.sum(np.multiply(z_concat, np.array(self.z)), axis=1)
            q = q.reshape((self.batch_size, self.action_size), order='F')
            next_actions = np.argmax(q, axis=1)
            m_prob = [np.zeros((self.batch_size, self.atoms))
                      for _ in range(self.action_size)]
            for i in range(self.batch_size):
                if dones[i]:
                    T_zj = min(self.v_max, max(self.v_min, rewards[i]))
                    b_j  = (T_zj - self.v_min) / self.delta_z
                    l, u = math.floor(b_j), math.ceil(b_j)
                    m_prob[actions[i]][i][int(l)] += (u - b_j)
                    m_prob[actions[i]][i][int(u)] += (b_j - l)
                else:
                    for j in range(self.atoms):
                        T_zj = min(self.v_max, max(
                            self.v_min, rewards[i] + self.gamma * self.z[j]))
                        b_j = (T_zj - self.v_min) / self.delta_z
                        l, u = math.floor(b_j), math.ceil(b_j)
                        m_prob[actions[i]][i][int(
                            l)] += z_[next_actions[i]][i][j] * (u - b_j)
                        m_prob[actions[i]][i][int(
                            u)] += z_[next_actions[i]][i][j] * (b_j - l)

        main_value = self.dqn(states)[0]
        m_prob = torch.tensor(m_prob[0])
        loss = self.criterion(m_prob, main_value)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


    def _target_hard_update(self):
        if not self.Soft_Update:
            self.dqn_target.load_state_dict(self.dqn.state_dict())
        else:
            for target_param, param in zip(self.dqn_target.parameters(), self.dqn.parameters()):
                target_param.data.copy_(target_param.data * (1.0 - self.TAU) + param.data * self.TAU)

    def save(self, name):
        torch.save(self.dqn.state_dict(), name)

    def load(self, name):
        self.dqn.load_state_dict(torch.load(name))
        self.dqn_target.load_state_dict(torch.load(name))

# Main Training Loop
env_name = "CartPole-v0"
env = gym.make(env_name)

# parameters
hidden_size = 128
max_episodes = 200
batch_size = 64

# Exploration parameters
epsilon = 1.0
max_epsilon = 1.0
min_epsilon = 0.01
decay_rate = 0.025

atoms = 51
v_min = -10.
v_max = 10.

agent = DQNAgent(env, batch_size, 20, v_max, v_min, atoms)

for episode in range(max_episodes):
    state = env.reset()
    episode_reward = 0
    done = False

    while not done:
        action = agent.get_action(state, epsilon)
        next_state, reward, done, _ = env.step(action)
        agent.memory.put(state, action, reward, next_state, done)

        state = next_state
        episode_reward += reward

        if done:
            print(f"Episode: {episode + 1}/{max_episodes}, Score: {episode_reward}, Epsilon: {epsilon:.4f}")
            break

        if len(agent.memory) >= batch_size:
            sample = random.sample(agent.memory.memory, batch_size)
            states, actions, rewards, next_states, dones = map(np.array, zip(*sample))
            agent.train_step(states, actions, rewards, next_states, dones)

            if episode % agent.target_update == 0:
                agent._target_hard_update()

    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns 

Episode: 1/200, Score: 63.0, Epsilon: 1.0000
Episode: 2/200, Score: 18.0, Epsilon: 1.0000
Episode: 3/200, Score: 11.0, Epsilon: 0.9756
Episode: 4/200, Score: 40.0, Epsilon: 0.9517
Episode: 5/200, Score: 11.0, Epsilon: 0.9285
Episode: 6/200, Score: 22.0, Epsilon: 0.9058
Episode: 7/200, Score: 11.0, Epsilon: 0.8837
Episode: 8/200, Score: 13.0, Epsilon: 0.8621
Episode: 9/200, Score: 16.0, Epsilon: 0.8411
Episode: 10/200, Score: 34.0, Epsilon: 0.8205
Episode: 11/200, Score: 14.0, Epsilon: 0.8005
Episode: 12/200, Score: 27.0, Epsilon: 0.7810
Episode: 13/200, Score: 15.0, Epsilon: 0.7620
Episode: 14/200, Score: 16.0, Epsilon: 0.7434
Episode: 15/200, Score: 17.0, Epsilon: 0.7253
Episode: 16/200, Score: 16.0, Epsilon: 0.7076
Episode: 17/200, Score: 17.0, Epsilon: 0.6904
Episode: 18/200, Score: 18.0, Epsilon: 0.6736
Episode: 19/200, Score: 13.0, Epsilon: 0.6572
Episode: 20/200, Score: 22.0, Epsilon: 0.6413
Episode: 21/200, Score: 10.0, Epsilon: 0.6257
Episode: 22/200, Score: 9.0, Epsilon: 0.610

In [79]:
# IMPORTING LIBRARIES

import sys
IN_COLAB = "google.colab" in sys.modules

import random
import gym
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Softmax
from tensorflow.keras import optimizers, losses
from tensorflow.keras import Model
from collections import deque

from IPython.display import clear_output
import math

tf.keras.backend.set_floatx('float64')

class ReplayBuffer:
    def __init__(self, capacity=10000):
        self.memory = deque(maxlen=capacity)

    def put(self, state, action, reward, next_state, done):
        self.memory.append([state, action, reward, next_state, done])

    def sample(self):
        sample = random.sample(self.memory, batch_size)
        states, actions, rewards, next_states, done = map(
            np.asarray, zip(*sample))
        states = np.array(states).reshape(batch_size, -1)
        next_states = np.array(next_states).reshape(batch_size, -1)
        return states, actions, rewards, next_states, done

    def size(self):
        return len(self.memory)

# Neural Network Model Defined at Here.
class Network(Model):
    def __init__(self, state_size: int, action_size: int,
    z):
        """Initialization."""
        super(Network, self).__init__()

        self.state_size = state_size
        self.action_size = action_size
        self.atoms = atoms
        self.z = z

        # self.opt = tf.keras.optimizers.Adam(lr)
        self.model = self.create_model()

    def create_model(self):
        input_state = Input((self.state_size,))
        h1 = Dense(64, activation='relu')(input_state)
        h2 = Dense(64, activation='relu')(h1)
        outputs = []
        for _ in range(self.action_size):
            outputs.append(Dense(self.atoms, activation='softmax')(h2))
        return tf.keras.Model(input_state, outputs)

    def predict(self, state):
        return self.model.predict(state)

class DQNAgent:
    def __init__(
        self,
        env: gym.Env,
        batch_size: int,
        target_update: int,
    ):
        """Initialization.

        Args:
            env (gym.Env): openAI Gym environment
            memory_size (int): length of memory
            batch_size (int): batch size for sampling
            target_update (int): period for target model's hard update
            epsilon_decay (float): step size to decrease epsilon
            lr (float): learning rate
            max_epsilon (float): max value of epsilon
            min_epsilon (float): min value of epsilon
            gamma (float): discount factor
        """

        # CREATING THE Q-Network
        self.env = env
        self.env.seed(1234)
        self.state_size = self.env.observation_space.shape[0]
        self.action_size = self.env.action_space.n

        self.batch_size = batch_size
        # hyper parameters
        memory_size = 10000
        self.v_max = v_max
        self.v_min = v_min
        self.atoms = atoms
        self.delta_z = float(self.v_max - self.v_min) / (self.atoms - 1)
        self.z = [self.v_min + i * self.delta_z for i in range(self.atoms)]
        self.lr = 0.001
        self.target_update = target_update
        self.gamma = 0.99    # discount rate

        # create main model and target model
        self.dqn = Network(self.state_size, self.action_size, self.z
                          )
        self.dqn_target = Network( self.state_size, self.action_size, self.z
                          )
        self.train_start = 1000

        self.optimizers = optimizers.Adam(lr=self.lr, )
        self.criterion = tf.keras.losses.CategoricalCrossentropy()

        self.memory = ReplayBuffer(capacity=memory_size)
        self.Soft_Update = False # use soft parameter update

        self.TAU = 0.1 # target network soft update hyperparameter

        self._target_hard_update()

    # EXPLORATION VS EXPLOITATION
    def get_action(self, state, epsilon):
        state = np.reshape(state, [1, self.state_size])
        # 3. Choose an action a in the current world state (s)
        # If this number < greater than epsilon doing a random choice --> exploration
        if np.random.rand() <= epsilon:
            action = np.random.choice(self.action_size)

        ## Else --> exploitation (taking the biggest Q value for this state)
        else:
            action = self.get_optimal_action(state)

        return action

    def get_optimal_action(self, state):
        z = self.dqn.predict(state)
        z_concat = np.vstack(z)
        q = np.sum(np.multiply(z_concat, np.array(self.z)), axis=1)
        return np.argmax(q)

    # UPDATING THE Q-VALUE
    def train_step(self):
        states, actions, rewards, next_states, dones = self.memory.sample()
        z = self.dqn.predict(next_states)
        z_ = self.dqn_target.predict(next_states)
        z_concat = np.vstack(z)
        print(z_concat.shape)
        q = np.sum(np.multiply(z_concat, np.array(self.z)), axis=1)
        q = q.reshape((self.batch_size, self.action_size), order='F')
        next_actions = np.argmax(q, axis=1)
        m_prob = [np.zeros((self.batch_size, self.atoms))
                  for _ in range(self.action_size)]
        for i in range(self.batch_size):
            if dones[i]:
                T_zj = min(self.v_max, max(self.v_min, rewards[i]))
                b_j  = (T_zj - self.v_min) / self.delta_z
                l, u = math.floor(b_j), math.ceil(b_j)
                m_prob[actions[i]][i][int(l)] += (u - b_j)
                m_prob[actions[i]][i][int(u)] += (b_j - l)
            else:
                for j in range(self.atoms):
                    T_zj = min(self.v_max, max(
                        self.v_min, rewards[i] + self.gamma * self.z[j]))
                    b_j = (T_zj - self.v_min) / self.delta_z
                    l, u = math.floor(b_j), math.ceil(b_j)
                    m_prob[actions[i]][i][int(
                        l)] += z_[next_actions[i]][i][j] * (u - b_j)
                    m_prob[actions[i]][i][int(
                        u)] += z_[next_actions[i]][i][j] * (b_j - l)
        # self.dqn.train(states, m_prob)
        # def train(self, states, targets):
        m_prob = tf.stop_gradient(m_prob)
        dqn_variable = self.dqn.model.trainable_variables
        with tf.GradientTape() as tape:
            main_value = self.dqn.model(states)
            loss = self.criterion(m_prob, main_value)

        dqn_grads = tape.gradient(loss, dqn_variable)
        self.optimizers.apply_gradients(zip(dqn_grads, dqn_variable))

    # after some time interval update the target model to be same with model
    def _target_hard_update(self):
        if not self.Soft_Update:
            weights = self.dqn.model.get_weights()
            self.dqn_target.model.set_weights(weights)
            return
        if self.Soft_Update:
            q_model_theta = self.dqn.model.get_weights()
            dqn_target_theta = self.dqn_target.model.get_weights()
            counter = 0
            for q_weight, target_weight in zip(q_model_theta, dqn_target_theta):
                target_weight = target_weight * (1-self.TAU) + q_weight * self.TAU
                dqn_target_theta[counter] = target_weight
                counter += 1
            self.dqn_target.set_weights(dqn_target_theta)

    def load(self, name):
        self.dqn = load_model(name)

    def save(self, name):
        self.dqn.save(name)

# CREATING THE ENVIRONMENT
env_name = "CartPole-v0"
env = gym.make(env_name)

# parameters
target_update = 20


# INITIALIZING THE Q-PARAMETERS
hidden_size = 128
max_episodes = 200  # Set total number of episodes to train agent on.
batch_size = 64

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability
decay_rate = 0.025            # Exponential decay rate for exploration prob

atoms = 51
v_min = -10.
v_max = 10.

# train
agent = DQNAgent(
    env,
#     memory_size,
    batch_size,
    target_update,
#     epsilon_decay,
)

if __name__ == "__main__":

    update_cnt    = 0
    # TRAINING LOOP
    #List to contain all the rewards of all the episodes given to the agent
    scores = []

    # EACH EPISODE
    for episode in range(max_episodes):
        ## Reset environment and get first new observation
        state = agent.env.reset()
        episode_reward = 0
        done = False  # has the enviroment finished?


        # EACH TIME STEP
        while not done:
        # for step in range(max_steps):  # step index, maximum step is 200

            # 3.4.1 EXPLORATION VS EXPLOITATION
            # Take the action (a) and observe the outcome state(s') and reward (r)
            action = agent.get_action(state, epsilon)

            # 2.7.2 TAKING ACTION
            next_state, reward, done, _ = agent.env.step(action)
            agent.memory.put(state, action, reward, next_state, done)

            # Our new state is state
            state = next_state

            episode_reward += reward

            # if episode ends
            if done:
                scores.append(episode_reward)
                print("Episode: {}/{}, Score: {}, Epsilon: {:.4}".format(episode+1, max_episodes, episode_reward, epsilon))
                break
            # if training is ready
            if agent.memory.size() >= agent.batch_size:
                # 3.4.2 UPDATING THE Q-VALUE
                agent.train_step()
                update_cnt += 1

                # if hard update is needed
                if update_cnt % agent.target_update == 0:
                    agent._target_hard_update()

        # 2.8 EXPLORATION RATE DECAY
        # Reduce epsilon (because we need less and less exploration)
        epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns 

Episode: 1/200, Score: 16.0, Epsilon: 1.0
Episode: 2/200, Score: 10.0, Epsilon: 1.0
Episode: 3/200, Score: 19.0, Epsilon: 0.9756
2/2 [==============================] - 0s 6ms/step
(128, 51)
2/2 [==============================] - 0s 4ms/step
(128, 51)
2/2 [==============================] - 0s 6ms/step
(128, 51)
2/2 [==============================] - 0s 4ms/step
(128, 51)
Episode: 4/200, Score: 23.0, Epsilon: 0.9517
2/2 [==============================] - 0s 6ms/step
(128, 51)
2/2 [==============================] - 0s 7ms/step
(128, 51)
2/2 [==============================] - 0s 5ms/step
(128, 51)
2/2 [==============================] - 0s 6ms/step
(128, 51)
2/2 [==============================] - 0s 9ms/step
(128, 51)
2/2 [==============================] - 0s 9ms/step
(128, 51)
2/2 [==============================] - 0s 9ms/step
(128, 51)


KeyboardInterrupt: ignored